In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, col, lower, regexp_replace

spark = SparkSession.builder \
.appName('dpe_existants_rawdata_dl_job') \
.master('spark://spark-master:7077') \
.config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
.config('spark.ui.port', '4041') \
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/29 15:57:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [21]:
enedis = spark.read.csv('hdfs:///hadoop/dfs/data/enedis/raw_data/consommation-annuelle-residentielle_2021.csv',sep=';',header=True)
dpe = spark.read.parquet('hdfs:////hadoop/dfs/data/DPE/raw_data/dpe_logements_existants/lot/')
ban = spark.read.csv('hdfs:///hadoop/dfs/data/base-adresse-national/',sep=';',header=True)

# dpe_filtrer = dpe.filter(col('Code_postal_(BAN)').startswith('92'))
# enedis_filtrer = enedis.filter(col('code_commune').startswith('92'))

In [3]:
enedis.count()

407053

In [23]:
dpe.count()

87109

In [5]:
ban.count()

25423678

In [ ]:
# Normalisation des adresses
df = enedis.withColumn("adresse_brute", concat_ws(" ",
    col("adresse"),
    col("code_commune"),
    col("nom_commune")
))
enedis_normaliser = df.withColumn("adresse_brute", lower(regexp_replace(col("adresse_brute"), r"\s+", " ")))
distinct_values = enedis_normaliser.select("adresse_brute").distinct()
distinct_values.show(truncate=False)
df2 = dpe.withColumn("adresse_brute", concat_ws(" ",
    col('N°_voie_(BAN)'),
    col("Nom__rue_(BAN)"),
    col("Code_postal_(BAN)"),
    col("Nom__commune_(BAN)")
))
dpe_normaliser = df2.withColumn("adresse_brute", lower(regexp_replace(col("adresse_brute"), r"\s+", " ")))
distinct_values = dpe_normaliser.select("adresse_brute").distinct()
distinct_values.show(truncate=False)

# Création d'une colonne adresse normalisée dans le DataFrame de la BAN
ban_normalisee = ban.withColumn("adresse_normalisee", concat_ws(" ",
    col("numero"),
    col("rep"),
    col("nom_voie"),
    col("code_postal"),
    col("nom_commune")
))
ban_normaliser = ban_normalisee.withColumn("adresse_normalisee", lower(regexp_replace(col("adresse_normalisee"), r"\s+", " ")))
distinct_values = ban_normaliser.select("adresse_normalisee").distinct()
distinct_values.show(truncate=False)

# Création d'une colonne adresse normalisée dans le DataFrame de la BAN
ban_normaliser = ban_normaliser.withColumn("adresse_normalisee_insee", concat_ws(" ",
    col("numero"),
    col("rep"),
    col("nom_voie"),
    col("code_insee"),
    col("nom_commune")
))
ban_normaliser = ban_normaliser.withColumn("adresse_normalisee_insee", lower(regexp_replace(col("adresse_normalisee_insee"), r"\s+", " ")))
distinct_values = ban_normaliser.select("adresse_normalisee_insee").distinct()
distinct_values.show(truncate=False)

In [ ]:
ban_normaliser.show(1,vertical=True)

In [43]:
a = spark.read.parquet('hdfs:////hadoop/dfs/data/DPE/raw_data/dpe_logements_existants/lot_2/')
df2 = a.withColumn("adresse_brute", concat_ws(" ",
    col('N°_voie_(BAN)'),
    col("Nom__rue_(BAN)"),
    col("Code_postal_(BAN)"),
    col("Nom__commune_(BAN)")
))
dpe_normaliser = df2.withColumn("adresse_brute", lower(regexp_replace(col("adresse_brute"), r"\s+", " ")))

In [44]:
#join dpe normaliser avec la base ban 
dpe_join_ban = dpe_normaliser.join(ban_normaliser.select("adresse_normalisee", "id"), dpe_normaliser.adresse_brute == ban_normaliser.adresse_normalisee, how="inner")
distinct_values = dpe_join_ban.select("id").distinct()
distinct_values.show(truncate=False)

+----------------+
|id              |
+----------------+
|92004_4130_00008|
|92002_8115_00003|
|92063_2890_00012|
|92063_2890_00088|
|92049_8155_00004|
|92050_4140_00334|
|92044_1451_00102|
|92044_8141_00006|
|92024_4420_00044|
|92020_6185_00013|
|92019_6146_00004|
|92004_0670_00001|
|92078_4500_00014|
|92051_8880_00019|
|92050_6205_00006|
|92044_7140_00014|
|92026_8130_00020|
|92025_4442_00156|
|92012_1425_00082|
|92012_4110_00093|
+----------------+
only showing top 20 rows



In [45]:
dpe_join_ban.count()

59999

In [46]:
dpe_join_ban.write.mode('overwrite').partitionBy('Année').format('parquet') \
    .option("path", "hdfs:///hadoop/dfs/data/DPE/staging/dpe_logements_existants/lot_2/") \
    .saveAsTable("dpe_logements_existants_normalized")

In [47]:
#join enedis normaliser avec la base ban 

enedis_join_ban = enedis_normaliser.join(ban_normaliser.select("adresse_normalisee_insee", "id"), enedis_normaliser.adresse_brute == ban_normaliser.adresse_normalisee_insee, how="left")
distinct_values = enedis_join_ban.select("adresse_brute").distinct()
distinct_values.show(truncate=False)


+-------------------------------------------------------+
|adresse_brute                                          |
+-------------------------------------------------------+
|1 rue de la mairie 69205 saint-genis-les-ollières      |
|5 avenue foch 59378 marcq-en-barœul                    |
|39 boulevard baptiste bonnet 13055 marseille           |
|7 traverse chante perdrix 13055 marseille              |
|97 allee de la rouguiere 13055 marseille               |
|12 rue guy de combaud roquebrune 13055 marseille       |
|2 rue marie de coulange 13055 marseille                |
|412 boulevard national 13055 marseille                 |
|128 boulevard notre dame 13055 marseille               |
|27 boulevard paul bouygues 13055 marseille             |
|171 avenue general leclerc 83118 saint-raphaël         |
|47 rue des deportes 44190 saint-sébastien-sur-loire    |
|1 rue des deportes 44190 saint-sébastien-sur-loire     |
|10 rue a briand et de la paix 42218 saint-étienne      |
|2 rue claude 

In [49]:
enedis_join_ban.count()

423262

In [53]:
dpe_join_ban.count()

59999

In [51]:
#supprimer des colonnes (!! rajouter tt les colonnes qui ne sont pas necessaire)
dpe_join_ban_cleaned = dpe_join_ban.drop('adresse_brute')

enedis_join_ban_cleaned = enedis_join_ban.drop('adresse_brute')

#jointure enedis et dpe sur l'id ban
enedis_dpe_joind= enedis_join_ban_cleaned.join(dpe_join_ban_cleaned, dpe_join_ban_cleaned.id == enedis_join_ban_cleaned.id, how="inner")

#faire le select apres la jointure pour garder un seul id ban
final_df = enedis_dpe_joind.select(
    enedis_join_ban.id,
    *[col for col in dpe_join_ban_cleaned.columns if col != "id"],
    *[col for col in enedis_join_ban_cleaned.columns if col != "id"]
)
final_df.show(1,truncate=False,vertical=True)

-RECORD 0-----------------------------------------------------------------------------------------------------------
 id                                                      | 10323_0690_00039                                         
 Conso_chauffage_dépensier_é_finale                      | 2971.3                                                   
 Emission_GES_ECS                                        | 159.8                                                    
 Type_énergie_n°1                                        | Électricité                                              
 Type_énergie_n°2                                        | NULL                                                     
 Nom__commune_(BAN)                                      | Romilly-sur-Seine                                        
 Coût_ECS_énergie_n°2                                    | NULL                                                     
 Emission_GES_chauffage                                  | 179.9

In [ ]:
# df = dpe_join_ban.filter(col('id').startswith('92002_8115'))
# final_df.count()
distinct_values = final_df.select("id").distinct()
distinct_values.show(truncate=False)

In [120]:
df = enedis_join_ban_cleaned.filter(col('id').startswith('92071_3561_00004'))
final_df.count()

1

In [ ]:
## netoyer le code et charger plus de données (voir comment faire pour télécharger la base des adresse total + le ficher parquet de enedis ) 

In [55]:
final_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- Conso_chauffage_dépensier_é_finale: string (nullable = true)
 |-- Emission_GES_ECS: string (nullable = true)
 |-- Type_énergie_n°1: string (nullable = true)
 |-- Type_énergie_n°2: string (nullable = true)
 |-- Nom__commune_(BAN): string (nullable = true)
 |-- Coût_ECS_énergie_n°2: string (nullable = true)
 |-- Emission_GES_chauffage: string (nullable = true)
 |-- Date_réception_DPE: string (nullable = true)
 |-- Coût_ECS_énergie_n°1: string (nullable = true)
 |-- Coût_total_5_usages: string (nullable = true)
 |-- Conso_ECS_é_finale: string (nullable = true)
 |-- Emission_GES_5_usages: string (nullable = true)
 |-- Code_postal_(BAN): string (nullable = true)
 |-- Conso_éclairage_é_finale: string (nullable = true)
 |-- Coût_refroidissement_dépensier: string (nullable = true)
 |-- Coordonnée_cartographique_X_(BAN): string (nullable = true)
 |-- Date_fin_validité_DPE: string (nullable = true)
 |-- Nombre_niveau_logement: string (nullable = true)


In [56]:
output_path = "/hadoop/dfs/data/dpe_join_enedis.csv"

# Écrire le DataFrame en tant que fichier CSV
final_df.write.csv(output_path, header=True, mode='overwrite')

24/08/29 18:33:48 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/08/29 18:33:48 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [122]:
spark.stop()